# Programación Entera: Heurísticas para el Problema de Asignación

## Descripción

[TODO]

## Autor
  
  * Sergio García Prado - [garciparedes.me](https://garciparedes.me)
  
## Fecha

  * Mayo de 2018


## Contenidos
  
  * [Problema de Asignación](#Problema-de-Asignación)
    * [Asignación Clásico](#Asignación-Clásico)
    * [Asignación Generalizado](#Asignación-Generalizado)

In [1]:
library(lpSolve)
library(magrittr)

In [2]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 200)

## Funciones de Apoyo

In [3]:
dataset.folder <- "./data"

In [4]:
ReadDataSet <- function() {
    
}

In [5]:
Solve <- function(direction = "max", f.obj, f.con, f.rhs, f.dir = rep("<=", nrow(f.con)), ...) {
    lp(direction, f.obj, f.con, f.dir, f.rhs, ...)
}

## Problema de Asignación

### Asignación Clásico

#### Descripción:
TODO

#### Modelo:
TODO

In [6]:
AssignmentExact <- function(cost, ...) {
    demand <- rep(1, dim(cost)[1])
    capacity <- rep(1, dim(cost)[2])
    m <- length(capacity)
    n <- length(demand) 
    sol <- list(solution = matrix(0, nrow = m, ncol = n), 
                objval = 0, slack = capacity)
    while (!all(demand == 0)) {
        ij <- which(cost == min(cost[sol$slack > 0, demand > 0]), arr.ind = TRUE)
        ij.slice <- which(sol$slack[ij[, 1]] > 0 & demand[ij[, 2]] > 0)[1]
        i <- ij[ij.slice, 1]
        j <- ij[ij.slice, 2]
        inc <- min(sol$slack[i], demand[j])    
        
        sol$solution[i, j] <- sol$solution[i, j] + inc
        sol$slack[i] <- sol$slack[i] - inc
        demand[j] <- demand[j] - inc
        sol$objval <- sol$objval + cost[i, j] * inc
    }
    return(sol)
}

#### Heurísticas:
TODO

In [7]:
AssignmentGreedy <- function(cost) {   
    demand <- rep(1, dim(cost)[1])
    capacity <- rep(1, dim(cost)[2]) 
    m <- length(capacity)
    n <- length(demand) 
    sol <- list(solution = matrix(0, nrow = m, ncol = n), 
                objval = 0, slack = capacity)
    while (!all(demand == 0)) {
        ij <- which(cost == min(cost[sol$slack > 0, demand > 0]), arr.ind = TRUE)
        ij.slice <- which(sol$slack[ij[, 1]] > 0 & demand[ij[, 2]] > 0)[1]
        i <- ij[ij.slice, 1]
        j <- ij[ij.slice, 2]
        inc <- min(sol$slack[i], demand[j])    
        
        sol$solution[i, j] <- sol$solution[i, j] + inc
        sol$slack[i] <- sol$slack[i] - inc
        demand[j] <- demand[j] - inc
        sol$objval <- sol$objval + cost[i, j] * inc
    }
    return(sol)
}

In [8]:
GreedyLocalSearchImprovement <- function(change.1, change.2, cost) {
    return((cost[change.1[, drop = FALSE]] + cost[change.2[, drop = FALSE]]) -
           (cost[change.1[1], change.2[2]] + cost[change.2[1], change.1[2]]))
}

In [9]:
GreedyLocalSearchBest <- function(cost, changes) {
    best = list(v = 0, i = 0, j = 0)
    for (i in 1:nrow(changes)) {
        for(j in 1:nrow(changes)) {
            v.temp <- GreedyLocalSearchImprovement(changes[i, ,drop = FALSE], 
                                                   changes[j, , drop = FALSE],
                                                   cost)
            if (best$v < v.temp) {
                best$v <- v.temp
                best$i <- i
                best$j <- j
            }
        }
    }
    return(best)
}

In [10]:
AssignmentGreedyLocalSearch <- function(cost) {    
    sol <- AssignmentGreedy(cost)
        
    changes <- which(sol$solution != 0, arr.ind = TRUE)
    best <- GreedyLocalSearchBest(cost, changes)
    while(best$v > 0) {

        sol$solution[changes[best$i, , drop = FALSE]] <- 0
        sol$solution[changes[best$j, , drop = FALSE]] <- 0
        
        temp <- changes[best$i, 2]
        changes[best$i, 2] <- changes[best$j, 2]
        changes[best$j, 2] <- temp
        
        
        sol$solution[changes[best$i, , drop = FALSE]] <- 1
        sol$solution[changes[best$j, , drop = FALSE]] <- 1
        
        sol$objval <- sol$objval - best$v
        
        best <- GreedyLocalSearchBest(cost, changes)
    }
    return(sol)
}

### Ejemplos

In [11]:
matriz.15x15.1 <- as.matrix(read.csv("./data/assignment/matriz_15x15_1.csv", header = FALSE))

In [12]:
a <- AssignmentExact(matriz.15x15.1)
matrix(a$solution, nrow = dim(matriz.15x15.1)[1])
a$objval

0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


V11 
218

In [13]:
AssignmentGreedy(matriz.15x15.1)

0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [14]:
AssignmentGreedyLocalSearch(matriz.15x15.1)

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [15]:
matriz.15x15.2 <- as.matrix(read.csv("./data/assignment/matriz_15x15_2.csv", header = FALSE))

In [16]:
a <- AssignmentExact(matriz.15x15.2)
matrix(a$solution, nrow = dim(matriz.15x15.2)[1])
a$objval

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


V2 
146

In [17]:
AssignmentGreedy(matriz.15x15.2)

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
AssignmentGreedyLocalSearch(matriz.15x15.2)

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
matriz.15x15.3 <- as.matrix(read.csv("./data/assignment/matriz_15x15_3.csv", header = FALSE))

In [20]:
a <- AssignmentExact(matriz.15x15.3)
matrix(a$solution, nrow = dim(matriz.15x15.3)[1])
a$objval

0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


V3 
205

In [21]:
AssignmentGreedy(matriz.15x15.3)

0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [22]:
AssignmentGreedyLocalSearch(matriz.15x15.3)

0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


### Asignación Generalizado

In [23]:
ReadGeneralizedAssignment <- function(file.name) {
    con <- 
        file.name %>%
        file(open = "r")
    raw <- 
        con %>%
        readLines(warn = FALSE) %>% 
        strsplit("[[:blank:]]") %>%
        unlist() %>%
        as.integer()
    close(con)
    list(m        = raw[1],
         n        = raw[2], 
         cost     = matrix(raw[3:(2 + raw[1] * raw[2])], 
                           nrow = raw[1], ncol = raw[2], byrow = TRUE),
         resource = matrix(raw[(3 + raw[1] * raw[2]):(2 + 2 * raw[1] * raw[2])], 
                           nrow = raw[1], ncol = raw[2], byrow = TRUE),
         capacity = raw[(3 + 2 * raw[1] * raw[2]):length(raw)]
        
    ) %>%
    return()
}

#### Descripción:
TODO

#### Modelo:
TODO

In [24]:
GeneralizedAssignmentExact <- function(problem.statement, direction = "max", ...) {    
    
    m <- problem.statement$m
    n <- problem.statement$n
    cost <- problem.statement$cost
    resource <- problem.statement$resource
    capacity <- problem.statement$capacity
    demand <- rep(1, n)
    
    
    conditions <- matrix(0, nrow = n + m, ncol = n * m)
    for (i in 1:m) {
        conditions[i, seq(n) + (i - 1) * n] <- resource[i, ]
    }
    for (i in (m + 1):(n + m)) {
        conditions[i, seq(i - m, m * n, by = n)] <- 1
    }
    res <- c(rep("<=", m), rep("=", n))
    return(Solve(direction, t(cost), conditions, c(capacity, demand), res, all.bin = TRUE, ...))
}

#### Heurísticas:
TODO

In [25]:
GeneralizedAssignmentGreedy <- function(p) {    
    sol <- list(solution = matrix(0, nrow = p$m, ncol = p$n), 
                objval = 0, slack = p$capacity)
    while(!all(colSums(sol$solution) == 1)) {
        max.resource <- max(sol$slack - p$resource[,colSums(sol$solution) == 0])
        
        ij <- which(sol$slack - p$resource == max.resource, arr.ind = TRUE)
        ij <- ij[colSums(sol$solution[, ij[,2], drop = FALSE]) == 0, , drop = FALSE]
        ij <- ij[which.max(p$cost[ij]), ,drop = FALSE]
        
        sol$solution[ij[1], ij[2]] <- 1
        sol$slack[ij[1]] <- sol$slack[ij[1]] - p$resource[ij[1], ij[2]]
        sol$objval <- sol$objval + p$cost[ij[1], ij[2]]
    }
    return(sol)
}

In [26]:
AssignmentGreedyLocalSearchImprovement <- function(change.1, change.2, cost) {
    return( - (cost[change.1[, drop = FALSE]] + cost[change.2[, drop = FALSE]]) +
              (cost[change.1[1], change.2[2]] + cost[change.2[1], change.1[2]]))
}

In [27]:
AssignmentGreedyLocalSearchBest <- function(p, changes, slack) {
    best = list(v = 0, i = 0, j = 0)
    for (i in 1:nrow(changes)) {
        for(j in 1:nrow(changes)) {
            if ((slack[changes[i, , drop = FALSE][1]] + p$resource[changes[i, , drop = FALSE]] - 
                     p$resource[changes[i, 1], changes[j, 2]] > 0) & 
                (slack[changes[j, , drop = FALSE][1]] + p$resource[changes[j, , drop = FALSE]] - 
                     p$resource[changes[j, 1], changes[i, 2]] > 0)) { 
                    
                    v.temp <- AssignmentGreedyLocalSearchImprovement(changes[i, ,drop = FALSE], 
                                                                     changes[j, , drop = FALSE],
                                                                     p$cost)
                    if (best$v < v.temp) {
                        best$v <- v.temp
                        best$i <- i
                        best$j <- j
                    }
                }
        }
    }
    return(best)
}

In [28]:
GeneralizedAssignmentGreedyLocalSearch <- function(p) {
    sol <- GeneralizedAssignmentGreedy(p)
    
    changes <- which(sol$solution != 0, arr.ind = TRUE)
    slack <- rowSums(sol$solution * p$resource)
    best <- AssignmentGreedyLocalSearchBest(p, changes, sol$slack)
    while(best$v > 0) {
        
        sol$solution[changes[best$i, , drop = FALSE]] <- 0
        sol$solution[changes[best$j, , drop = FALSE]] <- 0
        sol$slack[changes[best$i][1]] <- sol$slack[changes[best$i][1]] + p$resource[changes[best$i, , drop = FALSE]]
        sol$slack[changes[best$j][1]] <- sol$slack[changes[best$j][1]] + p$resource[changes[best$j, , drop = FALSE]]
        
        temp <- changes[best$i, 2]
        changes[best$i, 2] <- changes[best$j, 2]
        changes[best$j, 2] <- temp
        
        sol$solution[changes[best$i, , drop = FALSE]] <- 1
        sol$solution[changes[best$j, , drop = FALSE]] <- 1
        
        sol$slack[changes[best$i][1]] <- sol$slack[changes[best$i][1]] - p$resource[changes[best$i, , drop = FALSE]]
        sol$slack[changes[best$j][1]] <- sol$slack[changes[best$j][1]] - p$resource[changes[best$j, , drop = FALSE]]
        sol$objval <- sol$objval + best$v
        best <- AssignmentGreedyLocalSearchBest(p, changes, sol$slack)
    }
    return(sol)
}

### Ejemplos

In [29]:
gap2.1 <- ReadGeneralizedAssignment("./data/assignment/gap2_1.txt")

In [30]:
a <- GeneralizedAssignmentExact(gap2.1)
matrix(a$solution, nrow = gap2.1$m, byrow = TRUE)
a$objval

0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0


[1] 434

In [31]:
GeneralizedAssignmentGreedy(gap2.1)

0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0
0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0


In [32]:
GeneralizedAssignmentGreedyLocalSearch(gap2.1)

0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1
0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0
0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [33]:
gap4.1 <- ReadGeneralizedAssignment("./data/assignment/gap4_1.txt")

In [34]:
a <- GeneralizedAssignmentExact(gap4.1)
matrix(a$solution, nrow = gap4.1$m, byrow = TRUE)
a$objval

0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0
0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0
0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


[1] 656

In [35]:
GeneralizedAssignmentGreedy(gap4.1)

1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0
0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0
0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [36]:
GeneralizedAssignmentGreedyLocalSearch(gap4.1)

0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0
0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1


In [37]:
gap6.1 <- ReadGeneralizedAssignment("./data/assignment/gap6_1.txt")

In [38]:
a <- GeneralizedAssignmentExact(gap6.1)
matrix(a$solution, nrow = gap6.1$m, byrow = TRUE)
a$objval

0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0
0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


[1] 761

In [39]:
GeneralizedAssignmentGreedy(gap6.1)

1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [40]:
GeneralizedAssignmentGreedyLocalSearch(gap6.1)

0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [41]:
gap8.1 <- ReadGeneralizedAssignment("./data/assignment/gap8_1.txt")

In [42]:
# a <- GeneralizedAssignmentExact(gap8.1)
# matrix(a$solution, nrow = gap8.1$m, byrow = TRUE)
# a$objval

In [43]:
GeneralizedAssignmentGreedy(gap8.1)

0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1
0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


In [44]:
GeneralizedAssignmentGreedyLocalSearch(gap8.1)

0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [45]:
gap12.1 <- ReadGeneralizedAssignment("./data/assignment/gap12_1.txt")

In [46]:
# a <- GeneralizedAssignmentExact(gap12.1)
# matrix(a$solution, nrow = gap12.1$m, byrow = TRUE)
# a$objval

In [47]:
GeneralizedAssignmentGreedy(gap12.1)

0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
GeneralizedAssignmentGreedyLocalSearch(gap12.1)

1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0
0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
